# Продвинутый Python, семинар 9

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Так как мы проходим веб-разработку, то это явно не та вещь, которая делается через colab. Поэтому здесь написан просто код, который можно воспроизвести локально

## Jinja2

В прошлый раз мы разобрались с самым непонятный: сделали локальную БД, в которой будут все данные храниться. Теперь давайте к части с самим вебом!

Давайте для начала нарисуем наш сайт, как он должен выглядеть. Для этого надо немного больше погрузиться в разметку Jinja2. Что основного необходимо знать про это?

* {{ }} - через двойные скобки указываются переменные (которые мы можем указывать внутри функции render_template

* {% %} - в таких скобках указываются всякие условия (с помощью которых можно кастомизировать ваш шаблон

* {# #} - в таких скобках можно указывать комментарии

Помимо того, чтобы передавать сами переменные, также можно передавать функции!
Например, как сделать переадресацию без полного указания необходимой странички:

```
{{ url_for('login') }}  - здесь мы передаем функцию url_for, генерируя ссылку куда нам надо
```

Давайте сразу же разберем условия:

```
{% if not session.logged_in %} - если выполняется условие
(в данном случае параметр logged_in)

<show_a> - покажи вот это

{% else %} - Иначе

<show_b> - покажи это 

{% endif %} - заканчиваем
```

Помимо классического if-else есть и elif (само собой)

Можно не только условия делать, но можно и так же делать итерацию через for!

```
{% for value in values %}
    <p>value</p> - вывести все value в отдельных абзацах
{% endfor %}
```

### Наследование

Отдельный пункт - это наследование. Что это значит?

Опять-таки у вас есть страничка Вики ФКН. Вы видите, что у вас есть общие части (например header etc). Но мы же не хотим это прописывать для каждого сайта, верно?

Ровно поэтому есть наследование внутри Jinja2. Вы можете отдельно написать общую часть (которая отвечает отдельно за неизменяемые части, чаще всего это называется layout), а после просто добавлять другие блоки. Как это сделать?

Внутри родителя:

```
{% block body %}{% endblock %} - "резервируем" часть, которую может менять наследник
```

Внутри наследника:

```
{% extends "base.html" %} - указываем родителя, от чего наследуемся
{% block body %} - указываем часть, которую меняем
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
```

А еще поддерживается подобное наследование:

parent -> child -> grandchild

И если вам внутри child и grandchild вы меняете один и тот же блок (и хотите их оба вызвать), то тогда можно обращаться на уровень выше с помощью:

```
{{ super() }} - выведи то, что было на уровень выше

{{ super.super() }} - выведи то, что было на 2 уровня выше
```

### Фильтры

И последнее, о чем важно сказать для базового понимания - это фильтры. Скажем, что у вас есть переменная name, внутри которой строка "hi there"

А нам не нравится, что все с маленькой буквы (ну не дело это). А в коде забыли поменять (или не могли предугадать). Что же делать? Для этого внутри Jinja2 есть [фильтры](https://jinja.palletsprojects.com/en/3.1.x/templates/#builtin-filters):

```
{{ "%s, %s!"|format(greeting, name) }} - фильтр format, который работает как format в питоне

{{name | upper }} - сделать все КАПСОМ!
```

## Напишем странички!

Мы с вами напишем 3 страницы:

* layout.html - общая выкладка, которая общая для всех (где можно залогиниться)

* show_entries.html - показать твиты наши

* login.html - страница для того, чтобы залогиниться

### Шаг 1. layout.html

```
<!doctype html>

<title>Twitter</title>

<div class=page>
<h1>Twitter</h1>

<div class=metanav>

{% if not session.logged_in %}
    <a href="{{ url_for('login') }}">log in</a> {% else %}
    <a href="{{ url_for('logout') }}">log out</a> {% endif %}

</div>

<img src="{{ url_for('static', filename='meme.jpg') }}">

{% for message in get_flashed_messages() %}
    <div class=flash>{{ message }}</div>
{% endfor %}

{% block body %}{% endblock %}

</div>
```

### Шаг 2. login.html

```
{% extends "layout.html" %}
{% block body %}

<h2>Login</h2>

{% if error %}
    <p class=error><strong>Error:</strong> {{ error }}
{% endif %}

<form action="{{ url_for('login') }}" method=post>
<dl>
<dt>Username:
<dd><input type=text name=username>
<dt>Password:
<dd><input type=password name=password>
<dd><input type=submit value=Login>
</dl>
</form>

{% endblock %}
```

### Шаг 3. show_entries.html

```
{% extends "layout.html" %} {% block body %}

{% if session.logged_in %}

    <form action="{{ url_for('add_entry') }}" method=post class=add-entry>
    <dl>
    <dt>Title:
    <dd><input type=text size=30 name=title>
    <dt>Text:
    <dd><textarea name=text rows=5 cols=40></textarea> <dd><input type=submit value=Share>
    </dl>
    </form>
{% endif %}

<ul class=entries>

{% for entry in entries %}
    <li><h2>{{ entry.title }}</h2>{{ entry.text|safe }}
{% else %}
    <li><em>Unbelievable. No entries here so far</em>
{% endfor %}

</ul>

{% endblock %}
```

## А теперь опять к коду!

Какие методы у нас есть?

* Зайти на начальную страницу

* Логирование/разлогирование

* Показать все посты (это мы уже сделали)

* Сделать свой собственный пост (это сделали)

### Задание 1


Напишите функцию, которая приведет на главную страницу

In [ ]:
from flask import Flask, request, session, g, redirect, url_for, abort, render_template, flash

@app.route('/')
def hello_page():
    return render_template("layout.html")

### Задание 2

Напишите код для разлогирования (после разлогирования мы должны вывести плашку, что мы вышли, и также перевести на главный экран)

In [ ]:
@app.route('/logout')
def logout():
    session.pop('logged_in', None)
    flash('You were logged out, wow')
    return redirect(url_for('hello_page'))

### Задание 3

Напишите функцию, которая будет производить логирование

Что здесь важно?

Во-первых, у нас есть ошибка, которую надо будет отправлять для странички (что же не так, если пользователь ошибся)

Во-вторых, а как определить, что пользователь ввел то, что нужно, у нас же не было никакой регистрации? Для этого мы должны добавить конфигурации (добавим админа, у нас же тут пока сырой проект):

In [ ]:
import sqlite3

DATABASE = '/tmp/flaskr.db' 
DEBUG = True
SECRET_KEY = 'development key'
USERNAME = 'admin' 
PASSWORD = 'default'

app = Flask(__name__)
app.config.from_object(__name__)

app.config.update(dict(
    DATABASE=os.path.join(app.root_path, 'flaskr.db'),
    DEBUG=True,
    SECRET_KEY='development key',
    USERNAME='admin',
    PASSWORD='default'))

app.config.from_envvar('FLASKR_SETTINGS', silent=True)

In [ ]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != app.config['USERNAME']:
            error = 'Invalid username'
        elif request.form['password'] != app.config['PASSWORD']:
            error = 'Invalid password'
        else:
            session['logged_in'] = True
            flash('You were logged in')
            return redirect(url_for('show_entries'))
    return render_template('login.html', error=error)

## Соединяем и запускаем!

Странная фигня получилась, верно?..

Не хватает верстки (сделать бы шрифт другой, цвет, сделать resize картинки)..

Вот для этого всего есть CSS!

### CSS

CSS (Cascading Style Sheets) - это правила для внешнего вида нашего HTML-документа (которые мы видели абсолютно везде, на всех сайтах), без этого не было бы никакой красоты

Как можно задавать стиль? Структура любого CSS-документа выглядит как:

<для чего применить правило> - { <свойсто:значение; свойство:значение> }

Свойств достаточно [много](https://html5book.ru/css-spravochnik.html), давайте на примере разберемся, что внутри CSS документа:



```
Часть I - накидываем свойства на тэги

body { font-family: sans-serif; background: #eee; } 
a, h1, h2  { color: #377ba8; }
h1, h2 { font-family: 'Georgia', serif; margin: 0; }
h1 { border-bottom: 2px solid #eee; }
h2 { font-size: 1.2em; }
img {width: 35em;}

Часть II - накидываем свойства на классы (всегда с точкой)

.page { margin: 2em auto; width: 35em; border: 5px solid #ccc; padding: 0.8em; background: white; }
.entries  { list-style: none; margin: 0; padding: 0; }
.entries li {margin: 0.8em 1.2em}

Отдельный случай - здесь указывается форматирования для всех li внутри класса .entries


.entries li h2 { margin-left: -1em; }
.add_entry { font-size: 0.9em; border-bottom: 1px solid #ccc; }
.add_entry dl { font-weight: bold; }
.metanav { text-align: right; font-size: 0.8em; padding: 0.3em; margin-bottom: 1em; background: #fafafa;}
.flash { background: #cee5F5; padding: 0.5em; border: 1px solid #aacbe2; }
.border { background: #f0d6d6; padding: 0.5em; }

Если хотим для тэга с классом - то будет li.entries (как вариант)
```

А как решаются конфликты?

Допустим, что вы в одном месте указали, что текст должен быть красным, а в другом синим, кто победит?

Ответ: что позже идет в документе, то и выигрывает (на то оно и называется каскадированным)

И вот теперь соединив все это, вы получаете нужный результат! Победа!

## Животное дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Ambystoma_mexicanum_%286337857516%29.jpg/1920px-Ambystoma_mexicanum_%286337857516%29.jpg)

Сегодня у нас известное всем животное - амбистома (и аксолотль)

В чем разница? Аксолотль - это амбистомы до тех пор, пока они не станут взрослыми (то есть это лишь этап). Половозрелые амбистомы выглядят вот так:

![](https://upload.wikimedia.org/wikipedia/commons/0/00/Axolotl_ganz.jpg)

Аксолотль не может самостоятельно стать взрослой, потому что для взросления ей необходим тироксин, которые они сами вообще не вырабатывают (то есть взрослыми они становятся за счет внешних причин, например, засухи)

И для амбистом оставаться аксолотлем, видимо, максимально выгодно!

Аксолотли обладают фантастической способоностью к регенерации - они даже могут мозг регенирировать (не все части, но все-таки), не оставляя на себе никаких шрамов. Поэтому ученые их используют для изучения как идеальную модель

Но к сожалению они вымирают, потому что раньше ацтеки их ели, а потом пришли конкистадоры, естественно, испортили их среду обитания, да и сейчас ситуация еще хуже (из-за загрязнения воды в Мексике)
